In [1]:
import numpy as np

# Function to generate a random DNA sequence of a given length
def generate_random_sequence(length=100, nucleotides=['A', 'C', 'G', 'T']):
    return ''.join(np.random.choice(nucleotides, length))

# Function to break a DNA sequence into k-mers
def fragment_sequence_into_kmers(dna_sequence, k_size):
    return [dna_sequence[i:i+k_size] for i in range(len(dna_sequence) - k_size + 1)]

# Function to construct a directed graph from a list of k-mers
def construct_graph_from_kmers(kmer_list):
    edges = {}  
    in_degrees = {}  
    out_degrees = {}  

    for kmer in kmer_list:
        left_part = kmer[:-1] #prefix of the kmer
        right_part = kmer[1:] #suffix of the kmer

        # Ensure both the prefix and suffix are included in the graph
        if left_part not in edges:
            edges[left_part] = []
        if right_part not in edges:
            edges[right_part] = []

        # Create an edge from the prefix to the suffix
        edges[left_part].append(right_part)
        out_degrees[left_part] = out_degrees.get(left_part, 0) + 1
        in_degrees[right_part] = in_degrees.get(right_part, 0) + 1

    return edges, in_degrees, out_degrees

# Function to find the starting point of an Eulerian path in the graph
def find_starting_point(graph, in_degrees, out_degrees):
    start = end = None
    for node in set(in_degrees) | set(out_degrees):
        outs = out_degrees.get(node, 0)
        ins = in_degrees.get(node, 0)

        if outs > ins:
            start = node
        elif ins > outs:
            end = node

    if start and end:
        # Connect the end to the start node to make the graph Eulerian
        graph[end].append(start)
    return start

# Function to create an Eulerian path from the given graph
def create_eulerian_path(graph, start_node):
    # Helper function to find a new starting point for a path
    def find_new_start_point(path):
        for vertice in path:
            if graph[vertice]:
                return vertice
        return None
    # Return an empty path if no start node or graph is empty
    if not start_node or start_node not in graph:
        return []

    path = [start_node]
    # Construct the initial path from the start node
    while graph[start_node]:
        next_node = graph[start_node].pop()
        path.append(next_node)
        start_node = next_node
    # Extend the path by exploring unvisited edges
    while any(graph.values()):
        new_start = find_new_start_point(path)
        if not new_start:
            break

        path_extension = [new_start]
        while graph[new_start]:
            next_node = graph[new_start].pop()
            path_extension.append(next_node)
            new_start = next_node
        # Merge the new path into the main path
        merge_idx = path.index(new_start)
        path = path[:merge_idx] + path_extension + path[merge_idx+1:]

    return path[:-1] # Return the Eulerian path excluding the last artificial edge


# Function to reconstruct a sequence from an Eulerian path
def reconstruct_from_path(eulerian_path):
    reconstructed = eulerian_path[0]
    # Reconstruct the sequence by concatenating the last nucleotide of each k-mer
    for segment in eulerian_path[1:]:
        reconstructed += segment[-1]
    return reconstructed

# Main function to reconstruct the original sequence from a set of k-mers
def reconstruct_sequence_from_kmers(kmers):
    # Construct the graph and find the starting node
    graph, in_degrees, out_degrees = construct_graph_from_kmers(kmers)
    start_node = find_starting_point(graph, in_degrees, out_degrees)
    # Create the Eulerian path and reconstruct the sequence
    eulerian_path = create_eulerian_path(graph, start_node)
    return reconstruct_from_path(eulerian_path)

# Generate a random DNA sequence, fragment it into k-mers, shuffle the k-mers,
# and then attempt to reconstruct the original sequence
original_sequence = generate_random_sequence()
kmers = fragment_sequence_into_kmers(original_sequence, 20)
np.random.shuffle(kmers)
reconstructed_sequence = reconstruct_sequence_from_kmers(kmers)

# Print the original sequence, reconstructed sequence, and whether they match
print(original_sequence)
print(reconstructed_sequence)
print(original_sequence == reconstructed_sequence)






TTAGCACCTAAGTTCGGAGTCTGGCGTCTGAAGATAGCAAAGCGATACCAAGACGGGGGGAGGCGTCAGCGATCGCGATTCTAGGGACCGGCGGGAGATA
TTAGCACCTAAGTTCGGAGTCTGGCGTCTGAAGATAGCAAAGCGATACCAAGACGGGGGGAGGCGTCAGCGATCGCGATTCTAGGGACCGGCGGGAGATA
True


The constructed code uses a collection of k-mers and a simplified De Bruijn graph-based assembly technique to recreate the original string. It starts by creating k-mers from a DNA sequence that is created at random. After that, it constructs a representation of a De Bruijn graph, in which nodes stand in for (k-1)-mers and edges for k-mers that overlap. The goal of the procedure is to locate an Eulerian path—a path that travels around every edge in the graph precisely once—in it. This route lines up with the string that was rebuilt. The last character of the last node is appended, and the first character of each node in the Eulerian path is concatenated to complete the reconstruction. 